In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time

In [2]:
driver = webdriver.Chrome(ChromeDriverManager().install())


Looking for [chromedriver 80.0.3987.106 mac64] driver in cache 
File found in cache by path [/Users/barry/.wdm/drivers/chromedriver/80.0.3987.106/mac64/chromedriver]


In [3]:
# 20 job postings per page
def get_urls(url):
    
    nums = 25
    # get html back
    html = requests.get(url)
    html = html.text
    soup = BeautifulSoup(html, 'lxml')

    # add the common part between all search pages
    base_url = 'https://www.coursera.org/search?query=business%20analytics&indices%5Bprod_all_products_term_optimization%5D%5Bpage%5D=1&indices%5Bprod_all_products_term_optimization%5D%5Bconfigure%5D%5BclickAnalytics%5D=true&indices%5Bprod_all_products_term_optimization%5D%5Bconfigure%5D%5BruleContexts%5D%5B0%5D=en&indices%5Bprod_all_products_term_optimization%5D%5Bconfigure%5D%5BhitsPerPage%5D=10&configure%5BclickAnalytics%5D=true'
    urls = []
    urls.append(base_url)
    for i in range(2, nums):
        a = "https://www.coursera.org/search?query=business%20analytics&indices%5Bprod_all_products_term_optimization%5D%5Bpage%5D="+str(i)+'&indices%5Bprod_all_products_term_optimization%5D%5Bconfigure%5D%5BclickAnalytics%5D=true&indices%5Bprod_all_products_term_optimization%5D%5Bconfigure%5D%5BruleContexts%5D%5B0%5D=en&indices%5Bprod_all_products_term_optimization%5D%5Bconfigure%5D%5BhitsPerPage%5D=10&configure%5BclickAnalytics%5D=true'
        urls.append(a)
        
    return urls

In [4]:
urls = get_urls('https://www.coursera.org/search?query=business%20analytics&')

In [6]:
course_links, course_desc = [],[]
# loop over all page-urls
for page_url in urls:

    # get the HTML of the search results page
    page = requests.get(page_url)
    content = page.text
    soup = BeautifulSoup(content, 'lxml')

    results = soup.find_all('li',{'class':'ais-InfiniteHits-item'})

    for course in results:

        course_link = 'https://www.coursera.org' + course.find('a')['href']
        
        #description
        course = requests.get(course_link)
        course_soup = BeautifulSoup(course.text, 'lxml')
        try:
            desc = course_soup.find('div',{'class':'description'}).text
        except:
            desc = 'No Descriptions'
        course_desc.append(desc)
        
        course_links.append(course_link)

In [7]:
#chapter links
course_len,course_names,chapter_names,chapter_links = [],[],[],[]
for link in course_links:
    
    page = requests.get(link)
    content = page.text
    soup = BeautifulSoup(content, 'lxml')
    
    course_names.append(soup.find('h1').text)
    #click buttons and generate full_link to further operations
    driver.get(link)
    time.sleep(5)
    try:
        button = driver.find_element_by_xpath('//*[@id="main"]/div/div[6]/div/div/div[2]/ul/li/div[2]/button/span')
        button.click()
        all_link = driver.page_source
    except:
        all_link = driver.page_source
    
    course_soup = BeautifulSoup(all_link, 'lxml')
    
    #course chapters names
    chapters = course_soup.find_all('h3',{'class':"headline-3-text bold m-t-1 m-b-2"})
    for i in chapters:
        chapter_names.append(i.text)
    
    #chapter links
    a = course_soup.find_all('div',{'class':'Row_nvwp6p CourseItem'})
    course_len.append(len(a))
    for b in a:
        chapter_links.append('https://www.coursera.org' + b.find('a',{'data-e2e':"course-link"})['href'])

In [8]:
len(chapter_names)

242

In [9]:
len(chapter_links)

242

In [10]:
#section names and description
section_len = []

for link in chapter_links:
    driver.get(link)
    time.sleep(5)
    try:
        buttons = driver.find_element_by_xpath('//*[@id="expand_module_description_button"]')
        buttons.click()
        full_link = driver.page_source
    except:
        full_link = driver.page_source
        
    chapter_soup = BeautifulSoup(full_link, 'lxml')
    
    #names and len
    a = chapter_soup.find_all('h2',{'class':'headline-2-text bold m-b-2'})
    section_len.append(len(a))

In [11]:
section_names,section_desc = [],[]
for link in chapter_links:
    driver.get(link)
    time.sleep(5)
    try:
        buttons = driver.find_element_by_xpath('//*[@id="expand_module_description_button"]')
        buttons.click()
        full_link = driver.page_source
    except:
        full_link = driver.page_source
        
    chapter_soup = BeautifulSoup(full_link, 'lxml')
    
    a = chapter_soup.find_all('h2',{'class':'headline-2-text bold m-b-2'})
    
    #names
    for b in a:
        section_names.append(b.text)
    
    #desc
    c = chapter_soup.find_all('div',{'class':'content-inner'})[-len(a):]
    for d in c:
        section_desc.append(d.text)

In [12]:
len(section_desc)

1100

In [13]:
df_section = pd.DataFrame({'Section Names':section_names,'Section Descriptions':section_desc})
df_section

,Section Names,Section Descriptions
0,Introduction to Customer Analytics,What is Customer Analytics? How is this course...
1,Descriptive Analytics,"In this module, you’ll learn what data can and..."
2,Predictive Analytics,"Once you’ve collected and interpreted data, wh..."
3,Prescriptive Analytics,How do you turn data into action? In this modu...
4,Application/Case Studies,How do top firms put data to work? In this mod...
...,...,...
1095,"How to design reliable, scalable and maintaina...","Welcome to the session How to design reliable,..."
1096,Designing a transaccional system,"After completing this module, a learner will l..."
1097,Designing an analytical system,"After completing this module, a learner will l..."
1098,Designing an alternative to relational databases,"After completing this module, a learner will l..."


In [14]:
df_section.insert(1,'Section Length','')

In [15]:
df_section.insert(2,'Chapter Links','')

In [16]:
a = 0
for i in range(len(section_len)):
    num = section_len[i]
    link = chapter_links[i]
    df_section.iloc[a:a+num,1] = num
    df_section.iloc[a:a+num,2] = link
    a += num

In [17]:
df_section

,Section Names,Section Length,Chapter Links,Section Descriptions
0,Introduction to Customer Analytics,5,https://www.coursera.org/learn/wharton-custome...,What is Customer Analytics? How is this course...
1,Descriptive Analytics,5,https://www.coursera.org/learn/wharton-custome...,"In this module, you’ll learn what data can and..."
2,Predictive Analytics,5,https://www.coursera.org/learn/wharton-custome...,"Once you’ve collected and interpreted data, wh..."
3,Prescriptive Analytics,5,https://www.coursera.org/learn/wharton-custome...,How do you turn data into action? In this modu...
4,Application/Case Studies,5,https://www.coursera.org/learn/wharton-custome...,How do top firms put data to work? In this mod...
...,...,...,...,...
1095,"How to design reliable, scalable and maintaina...",6,https://www.coursera.org/learn/nosql-databases,"Welcome to the session How to design reliable,..."
1096,Designing a transaccional system,4,https://www.coursera.org/learn/data-intensive-...,"After completing this module, a learner will l..."
1097,Designing an analytical system,4,https://www.coursera.org/learn/data-intensive-...,"After completing this module, a learner will l..."
1098,Designing an alternative to relational databases,4,https://www.coursera.org/learn/data-intensive-...,"After completing this module, a learner will l..."


In [18]:
df_course = pd.DataFrame({'Course Names':course_names,'Course Length':course_len,'Course Descriptions':course_desc})

In [19]:
df_chapter = pd.DataFrame({'Chapter Names':chapter_names,'Chapter Links':chapter_links})

In [20]:
df_chapter.insert(0,'Course Names','')
df_chapter.insert(1,'Course Length','')
df_chapter.insert(2,'Course Descriptions','')

In [21]:
a = 0
for i in range(len(df_course)):
    num = df_course.iloc[i,1]
    df_chapter.iloc[a:a+num,0] = df_course.iloc[i,0]
    df_chapter.iloc[a:a+num,1] = df_course.iloc[i,1]
    df_chapter.iloc[a:a+num,2] = df_course.iloc[i,2]
    a += num

In [22]:
df_chapter

,Course Names,Course Length,Course Descriptions,Chapter Names,Chapter Links
0,Business Analytics Specialization,5,This Specialization provides an introduction t...,Customer Analytics,https://www.coursera.org/learn/wharton-custome...
1,Business Analytics Specialization,5,This Specialization provides an introduction t...,Operations Analytics,https://www.coursera.org/learn/wharton-operati...
2,Business Analytics Specialization,5,This Specialization provides an introduction t...,People Analytics,https://www.coursera.org/learn/wharton-people-...
3,Business Analytics Specialization,5,This Specialization provides an introduction t...,Accounting Analytics,https://www.coursera.org/learn/accounting-anal...
4,Business Analytics Specialization,5,This Specialization provides an introduction t...,Business Analytics Capstone,https://www.coursera.org/learn/wharton-capston...
...,...,...,...,...,...
237,Fundamentos Empresariales Specialization,4,"En esta EspecializaciÃ³n, desarrollarÃ¡s tÃ©cn...",Introducción a las Finanzas Corporativas,https://www.coursera.org/learn/wharton-finanza...
238,Database systems Specialization,4,This specialized program is aimed at computer ...,Relational database systems,https://www.coursera.org/learn/relational-data...
239,Database systems Specialization,4,This specialized program is aimed at computer ...,Business intelligence and data warehousing,https://www.coursera.org/learn/business-intell...
240,Database systems Specialization,4,This specialized program is aimed at computer ...,NoSQL systems,https://www.coursera.org/learn/nosql-databases


In [25]:
df_section.to_csv('section_topics_ba.csv')

In [26]:
df_chapter.to_csv('chapter_topics_baa.csv')